In [8]:
from utilities import init_bigquery_client
from google.cloud import bigquery
import os
import pandas as pd
import numpy as np
import plotly.express as px

#init BigQuery client
bq = init_bigquery_client()

Using BigQuery credentials: etl-testing-478716-c0b6c2c512e0.json


## Bulk Query

In [3]:
# Read from the 'user' table in BigQuery
query = """
    SELECT *
    FROM `etl-testing-478716.firebase_etl_prod.users`
"""
users_df = bq.query(query).to_dataframe()

# Read from the 'friends' table in BigQuery
query = """
    SELECT *
    FROM `etl-testing-478716.firebase_etl_prod.friends`
"""
friends_df = bq.query(query).to_dataframe()


/opt/miniconda3/envs/heyyall/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


## Friend Graph

In [19]:
#creating dataframe with parent_id and friend_id info merged
merged_df = friends_df.merge(users_df, left_on='parent_id', right_on='user_id', how='left', suffixes=('_friend', '_user'))
merged_df = merged_df.rename(columns={'fullName': 'fullName_parent', 'username': 'username_parent'})

#merge to get friend info
merged_df = merged_df.merge(users_df, left_on='friend_id', right_on='user_id', how='left', suffixes=('', '_user2'))

merged_df = merged_df.rename(columns={'fullName': 'fullName_friend', 'username': 'username_friend',
                                       'businessUser': 'businessUser_parent', 'businessUser_user2': 'businessUser_friend',
                                       'contactAccessGranted': 'contactAccessGranted_parent', 'contactAccessGranted_user2': 'contactAccessGranted_friend'})

#cleaning columns
columns_to_keep = ['parent_id', 'friend_id', 'fullName_parent', 'username_parent', 'businessUser_parent',
                   'contactAccessGranted_parent', 'fullName_friend', 'username_friend', 'businessUser_friend',
                   'contactAccessGranted_friend']
merged_df = merged_df.drop(columns=[x for x in merged_df.columns if x not in columns_to_keep])
#dropping rows with missing names (user may have deleted account)
merged_df = merged_df.dropna(subset=['fullName_parent', 'fullName_friend'])

In [21]:
merged_df

,parent_id,friend_id,fullName_parent,username_parent,contactAccessGranted_parent,businessUser_parent,fullName_friend,username_friend,contactAccessGranted_friend,businessUser_friend
0,0PNBuWC4P5by27fmCRYtT89Jdtl1,j9TDokuozyVw4cQIibWntIKL7iA3,trask dix,traskyall,False,False,Goodjuan,goodjuan,True,False
1,3dedM4sg6xZVoCDDkM5WWgDGhLd2,j9TDokuozyVw4cQIibWntIKL7iA3,Brandon Quick,brandonquick,True,False,Goodjuan,goodjuan,True,False
2,3dedM4sg6xZVoCDDkM5WWgDGhLd2,j9TDokuozyVw4cQIibWntIKL7iA3,Brandon Quick,brandonquick,True,False,Goodjuan,goodjuan,True,False
3,9gHK6nfBJrVyx1543vFgvOKL85N2,j9TDokuozyVw4cQIibWntIKL7iA3,Dane Iliff,dane,True,False,Goodjuan,goodjuan,True,False
4,HMNMQxE4ZlOqFietyb4bnTSHZEv2,C8m3vjY3F6cwmhVhwpnuhqYY0Je2,Danny Kopack,dannykopack,True,False,Michelle Prichard,michelleprichard,True,False
...,...,...,...,...,...,...,...,...,...,...
172,qi6Gzx0sBdesNixO4SWQaxG2psH2,QFxHzayNZSRyDcrLAPJf5Oi9dY42,Chris Weeks,weeks,True,False,Zach Giberson,zachgiberson,False,False
173,sBrSU6VZ1GePlQ6SyYrLRGEKtST2,QFxHzayNZSRyDcrLAPJf5Oi9dY42,Morgan Barnhill,morganbarnhill,True,False,Zach Giberson,zachgiberson,False,False
174,tJkyWxrUaIPB4OfGxjD3Pvr8evf1,QFxHzayNZSRyDcrLAPJf5Oi9dY42,Austin Jacobs,austinjacobs,False,False,Zach Giberson,zachgiberson,False,False
176,wJae35zVFGUkw8iGe25gRZCAz302,QFxHzayNZSRyDcrLAPJf5Oi9dY42,Bok,bok,True,False,Zach Giberson,zachgiberson,False,False
